In [6]:
import os
import json
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import ExtraDataToPOAMiddleware # Important for L2 networks like Base

# --- 1. Setup and Connection ---

# Load environment variables from .env file
load_dotenv()
quicknode_url = os.getenv("QUICKNODE_BASE_URL")

if not quicknode_url:
    raise Exception("QUICKNODE_BASE_URL not found in .env file. Please add it.")

# Initialize a Web3 instance with the QuickNode URL
w3 = Web3(Web3.HTTPProvider(quicknode_url))

# Inject PoA middleware, which is required for chains like Polygon, BSC, and Base
# This helps the web3.py library handle the block authoring mechanism of these chains
w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

# Check connection and verify we are on the correct chain (Base Mainnet ID is 8453)
if not w3.is_connected():
    raise Exception("Failed to connect to the Base network via QuickNode.")

chain_id = w3.eth.chain_id
if chain_id != 8453:
    raise Exception(f"Connected to wrong chain! Chain ID is {chain_id}, expected 8453 for Base.")

print(f"✅ Successfully connected to Base network (Chain ID: {chain_id})")


# --- 2. Define Event Signature ---

# The event signature for an ERC-20 Transfer is: Transfer(address,address,uint256)
# We need its Keccak-256 hash to use as the first topic (topic0) for filtering
TRANSFER_EVENT_TOPIC = '0x' + w3.keccak(text="Transfer(address,address,uint256)").hex()
print(f"Hashed Event Topic for 'Transfer': {TRANSFER_EVENT_TOPIC}")


# --- 3. Query for Logs ---

try:
    # Define the block range for the query.
    # QuickNode and other RPCs limit the range, so smaller chunks are better.
    latest_block = w3.eth.block_number
    from_block = latest_block - 100 # Query the last 100 blocks
    
    print(f"\n🔍 Querying for logs from block {from_block} to {latest_block}...")

    # Construct the filter for eth_getLogs
    # We are only filtering by the event signature (topic0) to get ALL ERC-20 transfers
    filter_params = {
        "fromBlock": from_block,
        "toBlock": latest_block,
        "topics": [TRANSFER_EVENT_TOPIC]
    }

    # Fetch the logs from the node
    logs = w3.eth.get_logs(filter_params)
    
    print(f"📊 Found {len(logs)} matching transfer events.")

    # --- 4. Process and Save Raw Data ---

    # The log objects from web3.py are not directly JSON serializable.
    # We need to convert them to a standard list of dictionaries.
    serializable_logs = []
    for log in logs:
        # Convert the AttributeDict to a regular dict
        log_dict = dict(log)
        
        # Convert bytes to hex strings for JSON compatibility
        log_dict['blockHash'] = log_dict['blockHash'].hex()
        log_dict['transactionHash'] = log_dict['transactionHash'].hex()
        log_dict['topics'] = [topic.hex() for topic in log_dict['topics']]
        
        serializable_logs.append(log_dict)

    # Define the output filename
    output_filename = "base_transfer_logs.json"

    # Write the raw data to a JSON file
    with open(output_filename, 'w') as f:
        json.dump(serializable_logs, f, indent=4)

    print(f"\n✅ Successfully saved raw log data to '{output_filename}'")

except Exception as e:
    print(f"\nAn error occurred: {e}")

✅ Successfully connected to Base network (Chain ID: 8453)
Hashed Event Topic for 'Transfer': 0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef

🔍 Querying for logs from block 33120178 to 33120278...
📊 Found 31102 matching transfer events.

An error occurred: Object of type HexBytes is not JSON serializable


In [10]:
whales = {
    "0xB604f2d512EaA32E06F1ac40362bC9157cE5Da96",
    "0xB604f2d512EaA32E06F1ac40362bC9157cE5Da96"
}

Working prototype: uses etherscan api to request different actions

In [9]:
import os
import requests
import json
from dotenv import load_dotenv

# --- 1. Setup ---

# Load environment variables from .env file
load_dotenv()
API_KEY = os.getenv("ETHERSCAN_API_KEY")

if not API_KEY:
    raise Exception("ETHERSCAN_API_KEY not found in .env file. Please add it.")

# etherscan v2 API endpoint URL
SCAN_URL = "https://api.etherscan.io/v2/api?chainid=8453"


WHALE_ADDRESS = "0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13"

# --- 2. Construct and Send API Request ---

# Parameters for the API request
# We are asking for the ERC-20 token transfers for the specified address

params = {
    "module": "account",
    "action": "tokentx",
    "address": WHALE_ADDRESS,
    "page": 1,
    "offset": 100,  # Get the 100 most recent transactions
    "sort": "desc", # 'desc' for latest transactions first
    "apikey": API_KEY
}

print(f"🔍 Fetching ERC-20 token transfers for address: {WHALE_ADDRESS}")

try:
    # Make the GET request to the Basescan API
    response = requests.get(SCAN_URL, params=params)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

    # --- 3. Process and Save the Data ---
    
    data = response.json()

    # Basescan API returns '1' if successful, '0' if not. Check the status.
    if data["status"] == "1":
        # The actual transaction data is in the 'result' key
        transactions = data["result"]
        print(f"✅ Found {len(transactions)} token transfers.")

        # Define the output filename
        output_filename = "whale_token_transfers.json"

        # Write the raw data to a JSON file
        with open(output_filename, 'w') as f:
            json.dump(transactions, f, indent=4)

        print(f"Successfully saved data to '{output_filename}'")

    else:
        # If status is '0', the 'result' key contains the error message
        print(f"❌ API Error: {data['message']}")
        print(f"   Result: {data['result']}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred with the network request: {e}")
except json.JSONDecodeError:
    print("Failed to decode JSON from the response. The API might be down or returning an invalid format.")



🔍 Fetching ERC-20 token transfers for address: 0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13
✅ Found 100 token transfers.
Successfully saved data to 'whale_token_transfers.json'


In [ ]:
import json
from collections import defaultdict


def analyze_interactions(json_data):
    '''
    simple data structure to count the number of times a given wallet interacts
    with different accounts
    Args: 
        json str or list
    Returns:
        dict: Dictionary with from_addresses as keys and interaction counts as values
        int: Total number of unique senders
    '''
    # Parse JSON
    if isinstance(json_data, str):
        transfers = json.loads(json_data)
    else:
        transfers = json_data

    #initialize hashmap
    sender_counts = defaultdict(int)

    for transfer in transfers:
        if not isinstance(transfer, dict):
            continue

        from_address = transfer.get("from", "").lower()

        if from_address:
            sender_counts[from_address] += 1
    return dict(sender_counts), len(sender_counts)

def print_interaction_summary(interactions, top_n=10):
    '''
    prints summary of interaction data
    Args:
        interactions (dict): Dictionary with addresses and their interaction counts
        top_n (int): Number of top interactors to show
    '''
    # Add error handling later in case of variable inputs

    total_interactions = sum(interactions.values())
    unique_senders = len(interactions)

    print(f"total transactions: {total_interactions}")
    print(f"Unique sender addresses: {unique_senders}")

    sorted_interactions = sorted(interactions.items(), key=lambda x: x[1], reverse=True)

    print(f"\nTop {min(top_n, len(sorted_interactions))} senders by frequency:")
    for i, (address, count) in enumerate(sorted_interactions[:top_n], 1):
        print(f"{i}. {address}: {count} transactions")

In [ ]:
with open('whale_token_transfers.json', 'r') as file:
    token_transfers = json.load(file)

# Analyze interactions
sender_interactions, unique_senders = analyze_interactions(token_transfers)
print_interaction_summary(sender_interactions)

# Look up specific sender (if needed)
specific_address = "0xae2fc483527b8ef99eb5d9b44875f005ba1fae13"
if specific_address.lower() in sender_interactions:
    print(f"\nAddress {specific_address} has sent {sender_interactions[specific_address.lower()]} transactions")

Found 54 unique 'from' addresses

Top 10 addresses by unique interactions:
1. 0x411d7d86bcf0a2a9d2e5c16aa4d01d9e28a3f55a: 1 unique interactions
2. 0xc2342be024ac5e5b66681942a7df62f0c1a61365: 1 unique interactions
3. 0xf977814e90da44bfa03b6295a0616a897441acec: 1 unique interactions
4. 0x9966dc02d7648380139f59265cca9b851a60e72e: 1 unique interactions
5. 0xf1ceb16d94083606db7f4d98400554f17125483b: 1 unique interactions
6. 0xad3b67bca8935cb510c8d18bd45f0b94f54a968f: 1 unique interactions
7. 0xad2f4724407eb5ef1c5e89f4bafe524a9b5b610d: 1 unique interactions
8. 0x498581ff718922c3f8e6a244956af099b2652b2b: 1 unique interactions
9. 0x3304e22ddaa22bcdc5fca2269b418046ae7b566a: 1 unique interactions
10. 0x888888888889758f76e7103c6cbf23abbf58f946: 1 unique interactions


IndexError: list index out of range